<h2>개인 구글 드라이브와 colab 연동 </h2>

In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


<h2>라이브러리 설치 </h2>

In [ ]:
!pip install sklearn-crfsuite

"""
sklearn에서 crfsuite만 install하고 싶은 경우
이 때 !는 local 컴퓨터해서 해당 명령어 실행하라는 의미
즉, local 컴퓨터에 pip가 있어야 함

단점은 구글 코랩 환경에서는 시간이 지나면 삭제가 됨. 그러면 재설치해야함
local에서는 삭제가 안됨
"""

'\nsklearn에서 crfsuite만 install하고 싶은 경우\n이 때 !는 local 컴퓨터해서 해당 명령어 실행하라는 의미\n즉, local 컴퓨터에 pip가 있어야 함\n\n단점은 구글 코랩 환경에서는 시간이 지나면 삭제가 됨. 그러면 재설치해야함\nlocal에서는 삭제가 안됨\n'

In [ ]:
"""
과제 실습 풀이 설명

해당 과제에서 바뀐 부분은 이 부분만 존재하고 나머지 부분은 동일합니다.
추가로 변형한 부분을 위주로 설명드리겠습니다.

우선 json과 파일 경로 관리를 위한 라이브러리를 import합니다.
그리고 우선 수업에서 다운로드하라는 데이터를 다운로드하고 그 중에서 5가지를 임의로 골라서 구글 드라이브 해당 파일과 동일한 위치에 data라는 폴더를 생성하고 그 안에 다 넣습니다.
IT_과학 도메인을 SC로 표현하고 비율을 0.1로 주고 문화(CU), 여성복지(WO), 교육(ED), 스포츠(SP)를 각각 비율을 적용하고 해당 파일 위치도 제공하는데 해당 도메인 내에 json 파일이 되게 많습니다.
그래서 해당 파일 경로는 우선 도메인 내의 모든 json 파일로 설정합니다.

한 json 파일에도 sentence가 되게 여러가지 존재해서 뽑아낼 수 있지만 저는 도메인 비율별로 즉, 1만개 데이터를 뽑게 되면 SC에서는 1000개의 json 파일을 추출해서 해당 도메인의 각 json 파일마다 sentence 1개를 추출해서 데이터로 모으는 방식입니다.

그 이후에 각 도메인 비율에 맞게 json 파일을 불러오고 해당 json 파일을 무작위로 선출합니다.
그리고 선출된 json 파일 각각마다 첫 번째 sentence를 추출해서 단어 단위로 추출하고 그 단어마다 B와 I를 알맞게 출력합니다.
그리고 해당 sentence를 기준으로 음절단위로 자르고 B와 I로 태그 표시한 것을 튜플을 이용해서  datas에 넣습니다.
이를 모든 json 폴더 경로에 대해서 모두 수행합니다.

모두 수행한 결과의 data를 한 번 섞습니다.

모두 적용했으면 90%는 train data로 사용합니다.
"""

import json
import glob
import random
import sklearn_crfsuite
from sklearn_crfsuite import metrics

# 도메인 비율 정의 (예: 도메인 A가 30%, 도메인 B가 40%, 도메인 C가 30%)
domain_ratios = {
    "SC": 0.1,
    "CU": 0.15,
    "WO": 0.25,
    "ED": 0.25,
    "SP": 0.25,
}

# JSON 파일이 있는 폴더의 경로 설정
folder_paths = {
    "SC": "/gdrive/MyDrive/Colab/Week06/data/SC/*.json",
    "CU": "/gdrive/MyDrive/Colab/Week06/data/CU/*.json",
    "WO": "/gdrive/MyDrive/Colab/Week06/data/WO/*.json",
    "ED": "/gdrive/MyDrive/Colab/Week06/data/ED/*.json",
    "SP": "/gdrive/MyDrive/Colab/Week06/data/SP/*.json",
}

# 데이터를 저장할 리스트
datas = []

# 각 도메인에 대해 JSON 파일을 불러오고 "sentence" 추출
for domain, ratio in domain_ratios.items():
    json_files = glob.glob(folder_paths.get(domain,""))  # 도메인에 해당하는 JSON 파일 목록 가져오기
    # 파일 수에 비례하도록 해당 도메인의 데이터 수 계산
    num_datas_to_select = int(ratio * 10000)

    # 데이터를 무작위로 선택 (num_datas_to_select를 데이터 리스트의 길이 미만으로 조정)
    selected_json_files = random.sample(json_files, num_datas_to_select)

    # 경로내 json 파일을 열고 작업 수행
    for json_file in selected_json_files:
        with open(json_file, "r", encoding="utf8") as file:
            data = json.load(file)

            #해당 json 파일에 대해서 첫 번째 sentence를 추출
            first_entity = data.get("named_entity", [{}])[0]
            first_sentence_obj = first_entity.get("title", [{}])[0] or first_entity.get("content", [{}])[0] or first_entity.get("subtitle", [{}])[0]
            first_sentence = first_sentence_obj.get("sentence", "")

            #추출한 문장을 단어 단위로 쪼개기
            words = first_sentence.split()
            tags = []
            # 각 단어에 대해 태그 할당(B,I)
            for word in words:
                # 첫 글자에 "B" 태그 할당
                tags.append("B")
                # 나머지 글자에 "I" 태그 할당
                for _ in range(len(word) - 1):
                    tags.append("I")

            #튜플로 대입 후 datas에 추가하기
            eumjeol_sequence, label = list(first_sentence.strip().replace(' ','')), tags
            datas.append((eumjeol_sequence, label))

# 데이터를 섞기 (무작위로 데이터를 섞음)
number_of_train_datas = int(len(datas)*0.9)

train_datas = datas[:number_of_train_datas]
test_datas = datas[number_of_train_datas:]

print("train_datas 개수 : " + str(len(train_datas)))
print("test_datas 개수 : " + str(len(test_datas)))

train_datas 개수 : 9000
test_datas 개수 : 1000


<pre>
<h2> 1. 문장의 각 음절을 crf 모델의 입력으로 사용 할 수 있도록 자질화 </h2>
  "BOS" : 시작 음절인지 여부
  "EOS" : 마지막 음절인지 여부
  "WORD" : 기준 음절
  "IS_DIGIT" : 기준 음절이 숫자인지 여부
  "-1_WORD" : 기준 음절의 왼쪽 첫번째 음절
  "-2_WORD" : 기준 음절의 왼쪽 두번째 음절
  "+1_WORD" : 기준 음절의 오른쪽 첫번째 음절
  "+2_WORD" : 기준 음절의 오른쪽 두번째 음절

  1.1 예시) ["나", "는", "사", "과", "가", "좋", "아"]
            ->
            [ { "BOS":True, "EOS":False, "WORD":"나", "IS_DIGIT":False, "+1_WORD":"는", "+2_WORD":"사" },
    { "BOS":False, "EOS":False, "WORD":"는", "IS_DIGIT":False, "-1_WORD":"나", "+1_WORD":"사", "+2_WORD":"과" },
    { "BOS":False, "EOS":False, "WORD":"사", "IS_DIGIT":False, "-2_WORD":"나", "-1_WORD":"는", "+1_WORD":"과", "+2_WORD":"가" }, ... ]
    
    나 -> { "BOS":True, "EOS":False, "WORD":"나", "IS_DIGIT":False, "+1_WORD":"는", "+2_WORD":"사" }
    는 -> { "BOS":False, "EOS":False, "WORD":"는", "IS_DIGIT":False, "-1_WORD":"나", "+1_WORD":"사", "+2_WORD":"과" }
    사 -> { "BOS":False, "EOS":False, "WORD":"사", "IS_DIGIT":False, "-2_WORD":"나", "-1_WORD":"는", "+1_WORD":"과", "+2_WORD":"가" }
    과 -> { "BOS":False, "EOS":False, "WORD":"과", "IS_DIGIT":False, "-2_WORD":"는", "-1_WORD":"사", "+1_WORD":"가", "+2_WORD":"좋" }
    가 -> { "BOS":False, "EOS":False, "WORD":"가", "IS_DIGIT":False, "-2_WORD":"사", "-1_WORD":"과", "+1_WORD":"좋", "+2_WORD":"아" }
    좋 -> { "BOS":False, "EOS":False, "WORD":"좋", "IS_DIGIT":False, "-2_WORD":"과", "-1_WORD":"가", "+1_WORD":"아" }
    아 -> { "BOS":False, "EOS":True, "WORD":"아", "IS_DIGIT":False, "-2_WORD":"가", "-1_WORD":"좋" }
<h2> 2. 자질화한 데이터와 해당 데이터의 라벨을 분리하여 각 리스트에 저장 </h2>
  학습 데이터 -> train_x(자질화한 데이터), train_y(각 데이터의 정답 라벨)에 저장
  평가 데이터 -> test_x(자질화한 데이터), test_y(각 데이터의 정답 라벨)에 저장
  
  2.1 예시)
    train_x -> [
    
    [ { "BOS":True, "EOS":False, "WORD":"나", "IS_DIGIT":False, "+1_WORD":"는", "+2_WORD":"사" },
    { "BOS":False, "EOS":False, "WORD":"는", "IS_DIGIT":False, "-1_WORD":"나", "+1_WORD":"사", "+2_WORD":"과" },
    { "BOS":False, "EOS":False, "WORD":"사", "IS_DIGIT":False, "-2_WORD":"나", "-1_WORD":"는", "+1_WORD":"과", "+2_WORD":"가" }, ... ],
    
    [ ... ],
    
    ...
    
    ]
    
    train_y -> [
    
    [ "B", "I", "B", "I", "I", "B", "I" ],
    
    [ ... ],
    
    ...
    
    
    ]

</pre>

In [ ]:
#sentence를 feature 변경하는 함수
"""
내가 관측하지 않은 값은 uniform하게 주기 -> 엔트로피의 최대화하게 설정
관측한 것은 그 확률 그대로 하게 할 것
결국 총합은 1이 되도록 유도

MEMM은 해당 state를 관련 feature로 확률을 구하는데 해당 feature에 앞뒤 문맥을 포함한 정보를 추가해서 방향성 유지
그런데 label bias 문제를 해결해야함. 이는 local하게 정규화하기에 발생
따라서 방향성을 없애서 CRFs로 해결!
"""
def sent2feature(eumjeol_sequence):
  features = []
  sequence_length = len(eumjeol_sequence)
  for index, eumjeol in enumerate(eumjeol_sequence):#enumerate로 인덱스와 해당 음절을 튜플로 가져옴
    #이제 feature를 모델링하자
    #예를들면 앞에 "이 있으면 B라고 할 수 있음
      feature = { "BOS":False, "EOS":False, "WORD":eumjeol, "IS_DIGIT":eumjeol.isdigit() }
      if (index == 0):
        feature["BOS"]=True
      elif (index == sequence_length-1):
        feature["EOS"]=True

      if (index-1 >= 0):
        feature["-1_WORD"] = eumjeol_sequence[index-1]#해당 음절 앞에 있는 것
      if (index-2 >= 0):
        feature["-2_WORD"] = eumjeol_sequence[index-2]#해당 음절 앞앞에 있는 것

      if (index+1 <= sequence_length-1):
              feature["+1_WORD"] = eumjeol_sequence[index+1]#해당 음절 뒤에 있는 것
      if (index+2  <= sequence_length-1):
              feature["+2_WORD"] = eumjeol_sequence[index+2]#해당 음절 뒤뒤에 있는 것

      features.append(feature)

  return features

#데이터 생성
train_x, train_y = [], []
for eumjeol_sequence, label in train_datas: #train data에서
    train_x.append(sent2feature(eumjeol_sequence))#해당 음절을 feature화해서 입력 데이터에 넣기
    train_y.append(label)

test_x, test_y = [], []
for eumjeol_sequence, label in test_datas:
    test_x.append(sent2feature(eumjeol_sequence))
    test_y.append(label)

<h2> 1. train_x, train_y를 이용하여 crf 모델 학습 </h2>

In [ ]:
#crf = sklearn_crfsuite.CRF(
#        algorithm='lbfgs',
#        c1=0.1,
#        c2=0.1,
#        max_iterations=100,
#        all_possible_transitions=True
#    )

#CRFs 학습
crf = sklearn_crfsuite.CRF()
try:
    crf.fit(train_x, train_y)
except AttributeError:
    pass

<pre>
<h2> 1. 학습한 모델을 test_x 데이터를 사용하여 평가 </h2>
<h2> 2. 성능 측정 </h2>
  2.1 metrics.flat_accuracy_score(x, y) 함수를 이용하여 성능 측정
    metrics.flat_accuracy_score(x, y)
  args
    x : 실제 정답 라벨이 있는 리스트
    y : 모델의 출력 라벨이 있는 리스트
  return :
    accuract 성능
  
<h2> 3. 모델의 출력 값과 정답 값을 이용하여 음절만으로 구성된 완전한 문장으로 변형 </h2>
  3.1 test_datas, pred_y 예시
    test_datas = [
    ( ["나", "는", "사", "과", "가", "좋", "아"], ["B", "I", "B", "I", "I", "B", "I"] ),
    
    ( ... ),
    
    ...
    
    ]
    
    pred_y = [
    
    ["B", "B", "B", "I", "I", "I", "I"],
    
    [ ... ],
    
    ...
    
    ]
    
    위의 문장을 기준으로한 변형 예시
    
    "나", "는", "사", "과", "가", "좋", "아" -> 나는 사과가 좋아 (정답 기준으로 변형)
    "나", "는", "사", "과", "가", "좋", "아" -> 나 는 사과가좋아 (모델 출력 기준으로 변형)
  
  3.2 최종 출력 예시
  
    정답 문장 : 1914- 18년의 전쟁은 인류를 통합시킨 최초의 공통분모였다.
    출력 문장 : 19 14- 18년의 전쟁은 인류를 통합시킨 최초의 공통 분모였다.

    정답 문장 : 하지만 이 전쟁은 죽음을 통해 인류를 통합시켰다.
    출력 문장 : 하지만이 전쟁은 죽음을 통해 인류를 통합시켰다.

    정답 문장 : 사라예보에서 한 세르비아인이 쏜 총 한발이 합스부르크가의 계승자를 죽였다.
    출력 문장 : 사라 예보에서 한세르비아인이 쏜총한 발이 합스부르크가의 계승자를 죽였다.
    
    ...
  
  
</pre>


In [ ]:
#CRFs 평가

#매개변수로 테스트 데이터와 예측한 데이터 넣기
def show_predict_result(test_datas, predict):
  for index_1 in range(len(test_datas)):
      eumjeol_sequence, correct_labels = test_datas[index_1]#정답 레이블인 gold label
      predict_labels = predict[index_1]#예측 레이블

      correct_sentence, predict_sentence = "", ""#문자열 초기화
      for index_2 in range(len(eumjeol_sequence)):#음절 시퀀스만큼 반복
          if(index_2 == 0):#첫 번째라는 것이기에 음절 시퀀스의 인덱스에 해당하는 것을 그대로 붙여넣기(assign)
              correct_sentence += eumjeol_sequence[index_2]
              predict_sentence += eumjeol_sequence[index_2]
              continue

          if(correct_labels[index_2] == "B"):#gold label이 B이면 앞에 space 추가하고 음절 그대로 붙이기
              correct_sentence += " "
          correct_sentence += eumjeol_sequence[index_2]

          if (predict_labels[index_2] == "B"):#예측 label이 B이면 앞에 space 추가하고 음절 그대로 붙이기
              predict_sentence += " "
          predict_sentence += eumjeol_sequence[index_2]

      print("정답 문장 : " + correct_sentence)
      print("출력 문장 : " + predict_sentence)
      print()

predict = crf.predict(test_x)#predict() 메서드로 평가 즉, CRFs 실행
#predict 변수에 예측 결과를 리턴

print("Accuracy score : " + str(metrics.flat_accuracy_score(test_y, predict)))
print()

print("10개의 데이터에 대한 모델 출력과 실제 정답 비교")
print()

show_predict_result(test_datas[:10], predict[:10])

Accuracy score : 0.9025812470844348

10개의 데이터에 대한 모델 출력과 실제 정답 비교

정답 문장 : (이름) 보내고 (이름)?… '텍사스 비전과 일치'(美 매체)
출력 문장 : (이름) 보내고 (이름)?…'텍사 스비전과 일치 '(美 매체)

정답 문장 : [올림픽] 8강전 장소도 고지대, (이름)호 예방주사 잘 맞았다
출력 문장 : [올림픽] 8강전장소도 고지대, (이름) 호예방 주사 잘 맞았다

정답 문장 : 롯데 (이름) 신발 보답 약속 '잘하고 나서 생각해볼게요'
출력 문장 : 롯데 (이름) 신발보 답약속 '잘하고 나서 생각해볼게요'

정답 문장 : ks 2차전 '주목!
출력 문장 : ks 2차전 '주목!

정답 문장 : [프로야구] po 1차전 매진…ps 7경기 중 6g 만원 관중
출력 문장 : [프로야구] po 1차 전매진…ps 7경기중 6g 만원 관중

정답 문장 : 평창조직위, 19일 야구장 찾아 '2018 평창 데이' 개최
출력 문장 : 평창 조직위, 19일 야구장 찾아' 2018 평창데이' 개최

정답 문장 : 넥센 '(이름) 악플러 고소?..
출력 문장 : 넥센' (이름) 악플러 고소?..

정답 문장 : 넥센, 롯데전 앞두고 (이름) 1군 말소 '홍성갑 첫 콜업'
출력 문장 : 넥센, 롯데전 앞두고 (이름) 1군 말소' 홍성갑 첫 콜업'

정답 문장 : '벼락 중거리슛‘ 강원 (이름), k리그1 8라운드 mvp
출력 문장 : '벼락 중거리슛 ‘강원 (이름), k리그 18라운드 mvp

정답 문장 : '고메스 결승골' 아르헨, 파라과이 1-0 격파…3경기 무패 이어가(종합)
출력 문장 : '고메스 결승골 '아르헨, 파라과이 1-0격파…3경기 무패이어가(종합)

